# Lab20: Association Analysis

In this lab we will be working on identifying *association rules*.  Association a way of identifying a combination of items that are associated with other items in the form of an IF-THEN type of rule:

    X -> Y

where X and Y are sets of items (goods, words, etc.).  The rule above asks if we purchase X then how often do we also purchase Y?

## Install mlxtend

We will be using the package `mlxtend` for our analysis.  This is a machine learning package (hence the ml) and has some features that overlap with SciKit Learn and other analysis pacakages.  

Using `Anaconda Navigator` install the `mlxtend` package.  It is on the `conda-forge` channel, so you will have to make sure that you have that set up as well.  (We did this back in Lab01.)

In [1]:
import pandas as pd

# TransactionEncoder will convert our data set to something
# the association-rule algorithm can use
from mlxtend.preprocessing import TransactionEncoder

# We will use the apriori module to generate a dataframe that
# we can use for association rule finding
from mlxtend.frequent_patterns import apriori

# We will use the association_rules module to generate
# our association rules from the apriori output data frame
from mlxtend.frequent_patterns import association_rules

## Load our groceries data set

This is from [Kaggle](https://www.kaggle.com/datasets/heeraldedhia/groceries-dataset?resource=download).  It is a CSV file containing around 38000 purchases of people from grocery stores.

Load and examine the file as a Pandas DataFrame.

Use the `sort_values` method to sort first by CustomerID and then by date, and examine the resulting sorted DataFrame. You will note that the file appears to contain daily transactions for each customer.  We will assume that a customer's daily purchases constitute a single transaction.

In [2]:
fn_grocery = 'Groceries_dataset.csv'
df_raw = pd.read_csv(fn_grocery)

In [3]:
df_raw=df_raw.sort_values(by=['Member_number','Date'])

## Group the transactions

Using `groupby` group each transaction by `Member_number` and `Date`.  This will give us one "row" with each purchase per customer per day.  We will use this in a moment to encode a new DataFrame for our `mlxtend`'s use.

In [4]:
df_raw=df_raw.groupby(['Member_number', 'Date'])


In [5]:
#待删除
df_raw.head()

,Member_number,Date,itemDescription
4843,1000,15-03-2015,sausage
8395,1000,15-03-2015,whole milk
20992,1000,15-03-2015,semi-finished bread
24544,1000,15-03-2015,yogurt
13331,1000,24-06-2014,whole milk
...,...,...,...
3578,5000,10-02-2015,soda
19727,5000,10-02-2015,root vegetables
34885,5000,10-02-2015,semi-finished bread
9340,5000,16-11-2014,bottled beer


## Encode the grouped data frame

Here we need to transform our dataset with the `Member_number` and `Date` index into a DataFrame that `mlxtend` can use to calculate the support of each transaction.

The `agg` method will return a DataFrame.  We need to convert its output into a list of lists.

In [5]:
def make_transaction_list(x):
    '''
    Will return a list of the unique items
    in a particular grouping when used with
    the agg method as its function
    '''
    return x.unique()


In [6]:
# Use the agg method and make_transaction_list to return a list of unique items for each transaction
unique_transaction = df_raw.agg(make_transaction_list)

In [7]:
#待删除

unique_transaction.head()

itemDescription
Member_number Date                                                         
1000          15-03-2015  [sausage, whole milk, semi-finished bread, yog...
              24-06-2014                  [whole milk, pastry, salty snack]
              24-07-2015                     [canned beer, misc. beverages]
              25-11-2015                        [sausage, hygiene articles]
              27-05-2015                         [soda, pickled vegetables]

## Convert to list of lists

From the DataFrame you created using the `agg` function, turn the output Series into a list of lists.

In [28]:
transactions=unique_transaction.values.tolist()

transaction_list=[]
for item in transactions:
    transaction_list.append(item[0].tolist())

In [30]:
#待删除

print(transaction_list[0:9])


[['sausage', 'whole milk', 'semi-finished bread', 'yogurt'], ['whole milk', 'pastry', 'salty snack'], ['canned beer', 'misc. beverages'], ['sausage', 'hygiene articles'], ['soda', 'pickled vegetables'], ['frankfurter', 'curd'], ['sausage', 'whole milk', 'rolls/buns'], ['whole milk', 'soda'], ['beef', 'white bread']]


## Encode the lists

Use the [`TransactionEncoder` method](http://rasbt.github.io/mlxtend/user_guide/preprocessing/TransactionEncoder/) from `mlxtend` to encode our list of lists into a matrix of presence/absence Boolean values.  See the example at the link above.

Once you do that, we can re-convert the output into a DataFrame by creating a dataframe based on the output and the value in the TransactionEncoder's `column_` attribute.  Again, see the link above.  It shows how to do this.

In [33]:
te = TransactionEncoder()
te_ary = te.fit(transaction_list).transform(transaction_list)

df_transaction = pd.DataFrame(te_ary, columns=te.columns_)


In [35]:
#待删除
df_transaction.head()

,Instant food products,UHT-milk,abrasive cleaner,artif. sweetener,baby cosmetics,bags,baking powder,bathroom cleaner,beef,berries,...,turkey,vinegar,waffles,whipped/sour cream,whisky,white bread,white wine,whole milk,yogurt,zwieback
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,True,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


## Use `apriori` to get support

The input to the association rule generator is a data frame that contains support for each transaction.  The [`aprori`](http://rasbt.github.io/mlxtend/user_guide/frequent_patterns/apriori/) module will do this for us.  See the link for documentation.

Let's use

    min_support=0.001  # All transactions that are > 1:1000

In [ ]:
# Before we begin, let's do a small
# amount of cleanup.  Let's remove all
# columns (items) that have a name less
# than three letters.  Use the Pandas
# drop method to perform this action.


#there is no columns' names less than 3 letters????
for item in df_transaction.columns:
    if len(item)<3: 
        print(item)

##df_transaction.drop(columns=['B', 'C'])

In [55]:
# Use apriori to create a dataframe with columns
# of support and itemset lists
df_apriori=apriori(df_transaction, min_support=0.001, use_colnames=True)

In [56]:
#待删除
df_apriori.head()

,support,itemsets
0,0.004010,(Instant food products)
1,0.021386,(UHT-milk)
2,0.001470,(abrasive cleaner)
3,0.001938,(artif. sweetener)
4,0.008087,(baking powder)


## Use association_rules to find the rules

Using the dataframe generated by `apriori`, find the association rules with the greatest lift.  See the [association_rules documentation](https://rasbt.github.io/mlxtend/api_modules/mlxtend.frequent_patterns/association_rules/) for how to do this.

Sort the resulting DataFrame by lift in descending order.  A lift > 1 indicates that the items are often purchased together and that buying X will increase the purchase of Y.  A lift of < 1 indicates the items are often substituted.  That is X is substituted for Y so X and Y don't appear together often.

Examine the resulting DataFrame.  For the association rule X -> Y, X is the column `antecedents` and Y is the column `consequents`.  If sorted you can see the metrics for each rule based upon the lift.

In [57]:
# Find the association rules
df_association = association_rules(df_apriori, metric='lift', min_threshold=0.8, support_only=False)

In [59]:
#待删除
df_association.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(bottled water),(UHT-milk),0.060683,0.021386,0.001069,0.017621,0.823954,-0.000228,0.996168
1,(UHT-milk),(bottled water),0.021386,0.060683,0.001069,0.050000,0.823954,-0.000228,0.988755
2,(other vegetables),(UHT-milk),0.122101,0.021386,0.002139,0.017515,0.818993,-0.000473,0.996060
3,(UHT-milk),(other vegetables),0.021386,0.122101,0.002139,0.100000,0.818993,-0.000473,0.975443
4,(sausage),(UHT-milk),0.060349,0.021386,0.001136,0.018826,0.880298,-0.000154,0.997391


In [60]:
# Sort the rules by lift
# and examine the output
# to find what rules were
# discovered
df_association.sort_values(by='lift',ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
732,(sausage),"(whole milk, yogurt)",0.060349,0.011161,0.001470,0.024363,2.182917,0.000797,1.013532
729,"(whole milk, yogurt)",(sausage),0.011161,0.060349,0.001470,0.131737,2.182917,0.000797,1.082219
730,"(sausage, whole milk)",(yogurt),0.008955,0.085879,0.001470,0.164179,1.911760,0.000701,1.093681
731,(yogurt),"(sausage, whole milk)",0.085879,0.008955,0.001470,0.017121,1.911760,0.000701,1.008307
247,(citrus fruit),(specialty chocolate),0.053131,0.015973,0.001403,0.026415,1.653762,0.000555,1.010726
...,...,...,...,...,...,...,...,...,...
572,(tropical fruit),(pastry),0.067767,0.051728,0.002807,0.041420,0.800735,-0.000699,0.989247
245,(rolls/buns),(citrus fruit),0.110005,0.053131,0.004678,0.042527,0.800423,-0.001166,0.988925
244,(citrus fruit),(rolls/buns),0.053131,0.110005,0.004678,0.088050,0.800423,-0.001166,0.975926
703,(other vegetables),"(whole milk, soda)",0.122101,0.011629,0.001136,0.009305,0.800165,-0.000284,0.997654
